# Code for Data Generation
by Loes & Merel

In [ ]:
import subprocess
import os
import numpy as np
import soundfile as sf
import random
import string
from numpy import ndarray
import pickle
import pandas as pd


In [ ]:
#variables
indir = 'TIMIT/' #directory of utterances
outdir = 'out/' 
secdir = 'splitdata/' #this is the actual out directory
normdir = 'normalized/' 
tempdir = 'temp/'
paddeddir = 'padded/'
labeldir = 'labels/'
n_wavfiles = 1000 #how many total overlapping files you want to generate
random_max = 7 #total amount of speakers in an overlapping file
mean = 0.4 #40% overlap
std = 0.01

In [ ]:
filenames=[]
for filename in os.listdir(indir):
    filenames.append(filename)

nrs = std * np.random.randn(n_wavfiles) + mean
label = []
new_files = []
commands = []
sec_files = [] 
splitcommands = []

## Creating the audio segments

In [ ]:
for f in range(n_wavfiles):
    
    n_files = random.randint(2, random_max) #nr of utterances per overlapping file
    #print("n_files = ", n_files)
    
    choices = np.random.choice(len(filenames),n_files) #random indices
    
    doubles = True
    while(doubles): #speakers may only speak once per file 
        names = []
        for c in choices:
            names.append(filenames[c][0:8]) #check entire speaker, not the particular file 
        
        if(len(names) != len(set(names))):
            choices = np.random.choice(len(filenames),n_files) 
        else:
            doubles = False
            
   # print('choices =', choices)
    new_filename = str(n_files)+"_"

    starttimes = [0]
    lengtes = []
    totstarttime = 0
    tempfilename = ""

    for i in range(len(choices)-1):
        file1 = filenames[choices[i]]
        file2 = filenames[choices[i+1]]

       # print(file1," & ", file2)
        new_filename += str(file1[0:10])+"_"
        
        in1f = sf.SoundFile(indir+file1)
        len1 = round((len(in1f) / in1f.samplerate),1) 
        in2f = sf.SoundFile(indir+file2)
        len2 = round((len(in2f) / in2f.samplerate),1) 
        
        overlap_time = round((len1+len2)*nrs[f],1)
        start_time = max(round(len1-overlap_time,1),0)
       # print('starttime = ', start_time)
    
        totstarttime += start_time
       # print('totstarttime', totstarttime)
    
        starttimes.append(totstarttime)
        lengtes.append(len1)    
        if(i == (len(choices)-2)): #for the last time
            lengtes.append(len2)
            new_filename += str(file2[0:10]) +".WAV"
    
        padcommand = "sox " + str(indir+file2) + " padded/" + str(file2) + " pad " + str(totstarttime) +" 0 "
       # print("padcommand == ", padcommand)
        commands.append(padcommand)
    
        if(i == 0):
            runcommand = "sox -m " + str(indir+file1) + " padded/" + str(file2) +" temp/tempfile"+str(i)+".WAV"
        else:
            runcommand = "sox -m temp/tempfile" + str(i-1) + ".WAV padded/" + str(file2) +" temp/tempfile"+str(i)+".WAV"
            
        #print("runcommand == ", runcommand)
        commands.append(runcommand)
    
      #  print()
      #  print(new_filename)
    normcommand = "sox temp/tempfile"+ str(len(choices)-2) + ".WAV " + str(normdir+new_filename)
   # print("normcommand == ", normcommand)
    commands.append(normcommand)
    
    #MAKING LABELS
  #  print('starttimes = ', starttimes)
  #  print('lengtes = ', lengtes)
    totaltime = int (round((starttimes[-1]+lengtes[-1]), 0))
  #  print('totaltime = ', totaltime)

    lijstje = [0] * totaltime
  #  print()
    starttime_index = 0

    for t in range(totaltime):
     #   print('t = ',t)
        sec_filename = ""
        for s in range(len(lengtes)):
            if(t+1 > (starttimes[s]) and t < (starttimes[s]+lengtes[s])):
                sec_filename += str(filenames[choices[s]][0:10]) + "_"
                lijstje[t] += 1
        sec_filename += str(t) + ".WAV"
      #  print('sec_filename = ', sec_filename)
      #  print('aantalspeakers = ', lijstje[t])
        sec_files.append([sec_filename, lijstje[t]])
        splitcommand = "sox "+str(normdir+new_filename)+" "+str(secdir+sec_filename)+" trim "+str(t)+" 1"
        splitcommands.append(splitcommand)
      #  print("splitcommand ==", splitcommand)
      #  print()
        
  #  print(lijstje)
    naam = "labels_" + str(new_filename)
  #  print(naam)
  #  print()

    new_files.append([naam, lijstje])

In [ ]:
df2 = pd.DataFrame(sec_files, columns = ['Filename', 'Speakers'])
df2.to_csv('secdata.csv', index=False)

for command in commands:
   # print(command)
    subprocess.run(command, shell=True)

In [ ]:
for scommand in splitcommands:
   # print(scommand)
    subprocess.run(scommand, shell=True)

In [ ]:
# DO NOT RUN THESE WITHOUT CHECKING!!!!!!
#Cleaningcommands:
"""
clean1 = "del /S temp "
clean2 = "del /S padded "
clean3 = "del /S ff2"
clean4 = "del /S normalized"
clean5 = "del /S splitdata"

cleancoms = [clean1, clean2, clean3, clean4]

for cc in cleancoms:
    print(cc)
    subprocess.run(cc, shell=True)
    subprocess.run("y", shell=True)
"""

In [ ]:
#POSITIVELY DO NOT RUN THIS WITHOUT CHECKING
#WILL DELETE YOUR SEGMENTS
"""
clean5 = "del /S splitdata"

subprocess.run(clean5, shell=True)
subprocess.run("y", shell=True)
"""

## Creating spectrograms
By Merel and Loes

In [ ]:
from scipy.signal import spectrogram
import os
from scipy.io import wavfile
from matplotlib import pyplot as plt
import stft
import matplotlib
import pandas as pd 

In [ ]:
directory = "splitdata/"
images = "spectograms/"

filenames=[]
for filename in os.listdir(directory):
    filenames.append(filename)

df = pd.read_csv("secdata.csv")

In [ ]:
#Creating and saving spectrograms
for f in range(len(filenames)):
   
    sample_rate, samples = wavfile.read(directory+filenames[f])
    
    ax = plt.axes([0, 0, 1, 1], frameon=False)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    spec = plt.specgram(samples,  Fs=sample_rate)
    
    filename = os.path.splitext(filenames[f])[0]+'.png'
    
    plt.savefig(images+filename, pad_inches=0.0, bbox_inches="tight")
    
    plt.clf()
    plt.close()